## 🔬 🧬 Parsing Bitcoin Price Data

### 📝 Imports

In [23]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import pandas as pd
import requests
import time
from datetime import datetime, timedelta

import yfinance as yf

import warnings
warnings.filterwarnings('ignore')

---

## 🔮 📝 Historical Data 2011-2020 in minutes

In [2]:
df_2011_2020 = pd.read_csv('data/bitstamp_cleaned.csv').drop(['Volume_(BTC)','Weighted_Price'],axis = 1)
df_2011_2020.columns = ['unix','date','open','high','low','close','volume']
df_2011_2020.date = df_2011_2020.unix.apply(datetime.fromtimestamp)

df_2011_2020.head()

,unix,date,open,high,low,close,volume
0,1325317920,2011-12-31 09:52:00,4.39,4.39,4.39,4.39,2.000000
1,1325346600,2011-12-31 17:50:00,4.39,4.39,4.39,4.39,210.720000
2,1325350740,2011-12-31 18:59:00,4.50,4.57,4.50,4.57,171.380338
3,1325350800,2011-12-31 19:00:00,4.58,4.58,4.58,4.58,41.220000
4,1325391360,2012-01-01 06:16:00,4.58,4.58,4.58,4.58,6.879160


In [3]:
df_2011_2020.shape

(3330541, 7)

In [4]:
df_2011_2020.date.agg(['min', 'max'])

min   2011-12-31 09:52:00
max   2020-09-14 03:00:00
Name: date, dtype: datetime64[ns]

In [5]:
timeStart_2020 = df_2011_2020.date.agg(['min', 'max'])[1]

---

## 👴🏻 🔮 Historical Data 2020.11.15 - 2022-03-17 in minutes

In [6]:
df_2020_2022 = pd.read_csv('data/BTS_USD_history_minute.csv').drop(['symbol','Volume BTC'],axis = 1)
df_2020_2022.columns = [*df_2020_2022.columns[:-1],'volume']
df_2020_2022.date = (df_2020_2022.unix/1000).apply(datetime.fromtimestamp)
df_2020_2022 = df_2020_2022.sort_values('unix').reset_index(drop = True)
df_2020_2022.head()

,unix,date,open,high,low,close,volume
0,1605472800000,2020-11-15 22:40:00,15850.000000,15853.000000,15849.0,15850.0,6931.354113
1,1605472860000,2020-11-15 22:41:00,15852.000000,15852.735754,15849.0,15850.0,7180.287699
2,1605472920000,2020-11-15 22:42:00,15853.389172,15854.000000,15853.0,15853.0,586.561000
3,1605472980000,2020-11-15 22:43:00,15854.344656,15857.000000,15853.0,15855.0,649.973000
4,1605473040000,2020-11-15 22:44:00,15855.000000,15856.000000,15855.0,15855.0,554.925000


In [7]:
df_2020_2022.shape

(691999, 7)

In [8]:
df_2020_2022.date.agg(['min', 'max']) 

min   2020-11-15 22:40:00
max   2022-03-17 08:12:00
Name: date, dtype: datetime64[ns]

In [9]:
timeEnd_2020 = df_2020_2022.date.agg(['min', 'max'])[0]
timeStart_2022 = df_2020_2022.date.agg(['min', 'max'])[1]

---

# 🧑🏻‍🔬 🏃 Coinbase Parsing

### 🔮 Defining Functions

In [10]:
def get_data(timeStart:str, barSize = 60)-> pd.DataFrame:
    
    apiUrl = 'https://api.pro.coinbase.com'
    sym = 'BTC-USD'
    delta = timedelta(minutes = barSize / 60)
    timeStart = pd.to_datetime(timeStart)
    timeEnd = timeStart + (300 * delta)
    
    parameters = {
    'start': timeStart,
    'end': timeEnd,
    'granularity': barSize
    }

    timeStart = timeStart.isoformat()
    timeEnd = timeEnd.isoformat()
    
    data = requests.get(f'{apiUrl}/products/{sym}/candles',
                     params = parameters,
                     headers = {'content-type':'application/json'}
                   )
    
    df = pd.DataFrame(data.json(),
                  columns = ['unix','low','high','open','close','volume']
                 )
    
    df['date'] = pd.to_datetime(df.unix,unit = 's')
    
    return df[['unix','date','open','high','low','close','volume']].sort_values('unix').reset_index(drop = True)

In [11]:
def fill_data(timeStart):
    data_len = 300
    df = pd.DataFrame()
    while data_len == 300:
        data = get_data(timeStart)
        df = pd.concat([df,data],ignore_index = True)
        data_len = data.shape[0]
        timeStart = data.date.max()
    return df

In [12]:
def fill_data_gap(timeStart,timeEnd):
    data_len = 300
    df = pd.DataFrame()
    while data_len == 300:
        if pd.to_datetime(timeStart) + (300 * timedelta(minutes = 1)) < pd.to_datetime(timeEnd):
            data = get_data(timeStart)
            df = pd.concat([df,data],ignore_index = True)
            data_len = data.shape[0]
            timeStart = data.date.max() 
        else:
            data = get_data(timeStart)
            data = data[data.date < pd.to_datetime(timeEnd)]
            df = pd.concat([df,data],ignore_index = True)
            return df

### 🧑🏻‍🔬 Experiments

In [13]:
timeStart = '2022-05-05 12:18:00'

fill_data(timeStart).shape

(8692, 7)

In [14]:
timeStart = '2022-05-05 12:18:00'
timeEnd = '2022-05-06 12:18:00'

df_test = fill_data_gap(timeStart,timeEnd)
df_test.shape

(1439, 7)

In [15]:
df_test.date.agg({'min','max'})

max   2022-05-06 12:17:00
min   2022-05-05 12:19:00
Name: date, dtype: datetime64[ns]

---

### 🔬 🧬 Filling Data

In [16]:
part1_2020 = fill_data_gap(timeStart_2020,timeStart_2020 + timedelta(days = 21))
part2_2020 = fill_data_gap(timeStart_2020 + timedelta(days = 22),timeStart_2020 + timedelta(days = 36))
part3_2020 = fill_data_gap(timeStart_2020 + timedelta(days = 37),timeStart_2020 + timedelta(days = 43))
part4_2020 = fill_data_gap(timeStart_2020 + timedelta(days = 44),timeStart_2020 + timedelta(days = 62))

df_2020 = pd.concat([part1_2020,part2_2020,part3_2020,part4_2020])

df_2020.head()

,unix,date,open,high,low,close,volume
0,1600052460,2020-09-14 03:01:00,10353.69,10356.03,10353.69,10356.03,0.447186
1,1600052520,2020-09-14 03:02:00,10356.12,10357.45,10356.12,10357.26,1.832700
2,1600052580,2020-09-14 03:03:00,10357.21,10357.45,10357.21,10357.45,1.451724
3,1600052640,2020-09-14 03:04:00,10357.45,10357.45,10357.19,10357.20,3.237631
4,1600052700,2020-09-14 03:05:00,10357.20,10357.45,10357.19,10357.45,1.104830


In [17]:
df_2022 = fill_data(timeStart_2022)
df_2022.tail()

,unix,date,open,high,low,close,volume
79490,1652274180,2022-05-11 13:03:00,29551.27,29551.49,29512.37,29531.27,4.147782
79491,1652274420,2022-05-11 13:07:00,29491.03,29553.99,29335.56,29456.54,258.530107
79492,1652274480,2022-05-11 13:08:00,29456.29,29518.29,29405.60,29497.13,56.480341
79493,1652274540,2022-05-11 13:09:00,29497.13,29510.62,29287.44,29408.41,76.659736
79494,1652274600,2022-05-11 13:10:00,29404.81,29475.34,29311.77,29376.22,36.674159


---

## 🪄 🧬 Putting in all together

In [18]:
bitcoin_price = pd.concat([df_2011_2020,df_2020,df_2020_2022,df_2022],ignore_index = True)
bitcoin_price.head()

,unix,date,open,high,low,close,volume
0,1325317920,2011-12-31 09:52:00,4.39,4.39,4.39,4.39,2.000000
1,1325346600,2011-12-31 17:50:00,4.39,4.39,4.39,4.39,210.720000
2,1325350740,2011-12-31 18:59:00,4.50,4.57,4.50,4.57,171.380338
3,1325350800,2011-12-31 19:00:00,4.58,4.58,4.58,4.58,41.220000
4,1325391360,2012-01-01 06:16:00,4.58,4.58,4.58,4.58,6.879160


In [19]:
bitcoin_price.tail()

,unix,date,open,high,low,close,volume
4186986,1652274180,2022-05-11 13:03:00,29551.27,29551.49,29512.37,29531.27,4.147782
4186987,1652274420,2022-05-11 13:07:00,29491.03,29553.99,29335.56,29456.54,258.530107
4186988,1652274480,2022-05-11 13:08:00,29456.29,29518.29,29405.60,29497.13,56.480341
4186989,1652274540,2022-05-11 13:09:00,29497.13,29510.62,29287.44,29408.41,76.659736
4186990,1652274600,2022-05-11 13:10:00,29404.81,29475.34,29311.77,29376.22,36.674159


In [26]:
bitcoin_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4186991 entries, 0 to 4186990
Data columns (total 7 columns):
 #   Column  Dtype         
---  ------  -----         
 0   unix    int64         
 1   date    datetime64[ns]
 2   open    float64       
 3   high    float64       
 4   low     float64       
 5   close   float64       
 6   volume  float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 223.6 MB


---

## 📝 Downloading CSV File

In [20]:
bitcoin_price.to_csv('data/bitcoin_price.csv')

---

## 🔮 🕵🏻‍♂️ Another Way

In [47]:
df = yf.Ticker('BTC-USD').history(
    period = '7d',
    interval = '1m'
)
df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2022-06-11 21:00:00+00:00,28435.460938,28435.460938,28435.460938,28435.460938,5130240,0,0
2022-06-11 21:01:00+00:00,28418.771484,28418.771484,28418.771484,28418.771484,0,0,0
2022-06-11 21:02:00+00:00,28414.580078,28414.580078,28414.580078,28414.580078,0,0,0
2022-06-11 21:03:00+00:00,28414.052734,28414.052734,28414.052734,28414.052734,380928,0,0
2022-06-11 21:04:00+00:00,28431.378906,28431.378906,28431.378906,28431.378906,1417216,0,0


In [48]:
df.reset_index().Datetime.agg(['min', 'max'])

min   2022-06-05 00:00:00+00:00
max   2022-06-11 21:04:00+00:00
Name: Datetime, dtype: datetime64[ns, UTC]

---